<a href="https://colab.research.google.com/github/shcheon95/ANN_TWS/blob/master/Copy_of_Code_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mat73

In [ ]:
import numpy as np 
import scipy.io as sio 
import mat73 
import torch from torch.autograd import Variable import 
torch.nn as nn import torch.nn.functional as F import torch.optim as optim from torch.utils.data import Dataset, DataLoader

import numpy as np
import scipy.io as sio
import mat73
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# mkdir /content/drive/My Drive/TWS_Recon/prac_pytorch/cp_best
# mkdir /content/drive/My Drive/TWS_Recon/prac_pytorch/cp_last

use_cuda = torch.cuda.is_available()
print(use_cuda)
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
## Making Data
mat_dum = mat73.loadmat('/content/drive/My Drive/TWS_Recon/prac_pytorch/mat_ann_input_TgmL_PgpcpL_N_ncstL_TrF2003T2014.mat')
reg_input = mat_dum.get('reg_input')
X_ = reg_input.get('pre_tr_2014');
t_pre = reg_input.get('t_pre')
X1 = np.transpose(X_)
y0 = reg_input.get('tar_tr')
y = y0[:,1]
y1 = np.transpose(y)


btsz = 1;
learning_rate = 0.001
epochs = 500
sel_mode = 2


class ANN_Tr_Dataset():
     def __init__(self,tar_pick):
mm

     def __getitem__(self,index):
         return self.x_data[index], self.y_data[index]

     def __len__(self):
         return self.len

class ANN_Ev_Dataset():
    def __init__(self,tar_pick):
        mat_dum = mat73.loadmat('/content/drive/My Drive/TWS_Recon/prac_pytorch/mat_ann_input_TgmL_PgpcpL_N_ncstL_TrF2003T2014.mat')
        reg_input = mat_dum.get('reg_input')
        X_ = reg_input.get('pre_pre_2015');
        y0 = reg_input.get('tar_p_2015')
        y = y0[:,tar_pick]
        self.len = X_.shape[0]
        self.x_data = torch.from_numpy(X_)
        self.y_data = torch.from_numpy(y)

    def __getitem__(self,index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len



sm_ = sel_mode - 1
dataset = ANNDataset(sm_)

train_loader = DataLoader(dataset=dataset,batch_size = btsz, shuffle=True)

# X = Variable(torch.from_numpy(X1), requires_grad=True)
# y = Variable(torch.from_numpy(y1))

if 0:
    print('type of X: ',type(X))
    print('shape of y: ',np.shape(y))
    print(reg_input.keys())
    type(reg_input)
    a = 5
    print('size of x: ',np.shape(X))
    #print(dum_dum)
    
## Model setting up
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(42,10)
#         self.fc2 = nn.Linear(10,10)
#         self.fc3 = nn.Linear(10,1)
#     def forward(self,x):
#         x = nn.LeakyReLU(self.fc1(x))
#         x = nn.LeakyReLU(self.fc2(x))
#         x = self.fc3(x)
#         return x
# net = Net()
net = nn.Sequential(
    nn.Linear(42,20),
    nn.ReLU(),
    nn.Linear(20,20),
    nn.ReLU(),
    nn.Linear(20,1)
)

net.cuda()



####>>>>>>>>>>>>>>>>>>>>>>>>>   Optimizer --------------------------------------
optimizer = optim.AdamW(net.parameters(),lr=learning_rate)





# create a loss function
criterion = nn.MSELoss()

loss_cri_ =100000000000;
# run the main training loop
for epoch in range(epochs):
    # learning_rate = learning_rate/(1+99*(epoch/epochs))
    cum_loss = torch.zeros(1,1)
    cum_loss.cuda()
    for batch_idx, data_ in enumerate(train_loader,0):
        # get the inputs
        pre, target = data_
        pre = pre.type(torch.cuda.FloatTensor)
        target = target.cuda()
        # wrap them in Variable
        pre, target = Variable(pre,requires_grad=True), Variable(target)

        # Forward pass:
        net_out = net(pre)

        # compute and print loss
        # loss를 구하기 전에 타입을 맞추자!!!!!
        net_out = net_out.float()
        target = target.type(torch.cuda.FloatTensor)
        loss = criterion(net_out, target)
        
        # print('loss: ',loss)
        # print(net_out.type(),net_out)
        # print(target.type(),target)

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # pre1 = torch.reshape(X[:,batch_idx],(1,42))
        # target = y[batch_idx]
        # pre = pre1.cuda()
        # target = target.cuda()

        # optimizer.zero_grad()

        # pre = pre.type(torch.cuda.FloatTensor)
        # pre= torch.reshape(pre,(1,42))
        # pre = Variable(pre)
        # net_out = net(pre)
        # net_out = net_out.float()
        # target = target.type(torch.cuda.FloatTensor)
        # loss = criterion(net_out, target)
        # loss.backward()
        # optimizer.step()
        # log_interval = 20
        cum_loss = cum_loss.cuda() + loss
    if epoch%100 == 0:
        print(epoch,'/',epochs)
        print('loss: ', loss.data)
        
    
    if cum_loss < loss_cri_:
        loss_cri_ = cum_loss
        checkpoint = {'model':net,'state_dict':net.state_dict(),
                      'optimizer':optimizer.state_dict()}
        torch.save(checkpoint,'/content/drive/My Drive/TWS_Recon/prac_pytorch/cp_best/best.py')
        print('Improved LOSS : ',loss_cri_.item())


# print(net_out,target)

In [ ]:
print(reg_input.keys())
print(reg_input.get('tar_p_2015').shape)

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False

    model.eval()
    return model

model = load_checkpoint('/content/drive/My Drive/TWS_Recon/prac_pytorch/cp_best/best.py')

In [ ]:
import forecasting_metrics as fm
import matplotlib.pyplot as plt
%matplotlib inline
import imageio
from sklearn.metrics import r2_score

print(reg_input.keys())
t_tr = reg_input.get('t_tr_2014')
y_ = dataset.y_data
x = dataset.x_data
print('y.shape: ',y_.shape)
print('x.shape: ',x.shape)
pred_ = torch.zeros(144,1)
for k in range(x.shape[0]):
    xin_ = x[k,:]
    # print(xin_.dtype)
    xin_ = np.reshape(xin_,(1,-1))
    # print('xin_.type: ',xin_.type)

    xin_ = xin_.type(torch.cuda.FloatTensor)
    # print(xin_.size())
    pred_[k] = model(xin_)
    # print(pred_[k].item(),y[k].item())

pred = pred_.detach().numpy()
# print(pred)
y_ = y_.numpy()
# pred_ = pred_.detach.numpy()

print(fm.nrmse(pred,y_))
print('r2: ', r2_score(pred,y_))
print(t_tr.shape)
print(y_.shape)
print(pred.shape)


plt.figure(figsize=(10,4))
plt.plot(t_tr,y_,'r',t_tr,pred,'k--')

In [ ]:
X1.shape[1]

In [ ]:
loss.item()

In [ ]:
pip install mat73


In [ ]:
!nvcc --version

In [ ]:
type(pre)